In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


team_acronyms = {
    "Arizona Cardinals": "ARI",
    "Atlanta Falcons": "ATL",
    "Baltimore Ravens": "BAL",
    "Buffalo Bills": "BUF",
    "Carolina Panthers": "CAR",
    "Chicago Bears": "CHI",
    "Cincinnati Bengals": "CIN",
    "Cleveland Browns": "CLE",
    "Dallas Cowboys": "DAL",
    "Denver Broncos": "DEN",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GB",
    "Houston Texans": "HOU",
    "Indianapolis Colts": "IND",
    "Jacksonville Jaguars": "JAX",
    "Kansas City Chiefs": "KC",
    "Las Vegas Raiders": "LV",
    "Los Angeles Rams": "LAR",
    "Los Angeles Chargers": "LAC",
    "Miami Dolphins": "MIA",
    "Minnesota Vikings": "MIN",
    "New England Patriots": "NE",
    "New Orleans Saints": "NO",
    "New York Giants": "NYG",
    "New York Jets": "NYJ",
    "Philadelphia Eagles": "PHI",
    "Pittsburgh Steelers": "PIT",
    "San Francisco 49ers": "SF",
    "Seattle Seahawks": "SEA",
    "Tampa Bay Buccaneers": "TB",
    "Tennessee Titans": "TEN",
    "Washington Commanders": "WSH"
}

url = "https://www.nfl.com/standings/league/2023/REG"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", class_="d3-o-table")

    teams = []
    wins = []
    losses = []
    acronyms = []

    for row in table.find_all("tr")[1:]:
        cols = row.find_all("td")
        team_name = cols[0].text.strip().split('\n')[0]
        team_wins = int(cols[1].text.strip())
        team_losses = int(cols[2].text.strip())

        # Use the dictionary to get the team acronym
        team_acronym = team_acronyms.get(team_name, "")
        
        teams.append(team_name)
        acronyms.append(team_acronym)
        wins.append(team_wins)
        losses.append(team_losses)

    df = pd.DataFrame({
        "Team Name": teams,
        "Team Acronym": acronyms,
        "Wins": wins,
        "Losses": losses
    })

    df['Win-Loss Difference'] = df['Wins'] - df['Losses']

    df.to_csv("NFL_Standings_2023.csv", index=False)

    print("Data has been scraped, processed, and saved to 'NFL_Standings_2023.csv'.")
else:
    print("Failed to retrieve the webpage.")


Data has been scraped, processed, and saved to 'NFL_Standings_2023.csv'.


In [5]:
score_values = [1.2, 1.1, 1, 0.9, 0.8, 0.7, 0.6]

scores = pd.DataFrame({"Scores": score_values})

afc_data = {
    "Adriana": ["KC", "BUF", "CIN", "JAX", "LAC", "BAL", "MIA"],
    "Alejandro": ["KC", "JAX", "NYJ", "CIN", "BUF", "BAL", "LAC"],
    "Boris": ["KC", "BAL", "BUF", "JAX", "CIN", "LAC", "NYJ"],
    "Caver": ["KC", "BUF", "CIN", "JAX", "BAL", "MIA", "LAC"],
    "Diego": ["KC", "BUF", "CIN", "JAX", "BAL", "LAC", "NYJ"],
    "Fernando": ["KC", "CIN", "BUF", "TEN", "BAL", "MIA", "LAC"],
    "Juan Pablo": ["KC", "BUF", "CIN", "JAX", "BAL", "NYJ", "MIA"],
    "Miguel Angel": ["CIN", "KC", "JAX", "NYJ", "BUF", "BAL", "DEN"],
    "Patricia": ["NYJ", "KC", "BAL", "TEN", "CIN", "LAC", "NE"]
}

afc_df = pd.DataFrame(afc_data, index=["Seed 1", "Seed 2", "Seed 3", "Seed 4", "Seed 5", "Seed 6", "Seed 7"])

nfc_data = {
    "Adriana": ["PHI", "SF", "DET", "NO", "DAL", "SEA", "ATL"],
    "Alejandro": ["SF", "PHI", "DET", "ATL", "DAL", "SEA", "GB"],
    "Boris": ["SF", "PHI", "MIN", "NO", "SEA", "DAL", "NYG"],
    "Caver": ["PHI", "SF", "MIN", "NO", "DET", "DAL", "SEA"],
    "Diego": ["PHI", "SF", "DET", "NO", "DAL", "MIN", "SEA"],
    "Fernando": ["PHI", "SF", "GB", "NO", "MIN", "WSH", "LAR"],
    "Juan Pablo": ["DAL", "SF", "NO", "DET", "PHI", "MIN", "SEA"],
    "Miguel Angel": ["PHI", "DET", "SF", "NO", "SEA", "CAR", "DAL"],
    "Patricia": ["PHI", "NO", "SEA", "MIN", "SF", "GB", "NYG"]
}


nfc_df = pd.DataFrame(nfc_data, index=["Seed 1", "Seed 2", "Seed 3", "Seed 4", "Seed 5", "Seed 6", "Seed 7"])

In [6]:
team_situation_df = pd.read_csv('NFL_Standings_2023.csv')

def calculate_total_scores(afc_df, nfc_df, team_situation_df, scores):
    total_scores = {}

    for person in afc_df.columns:
        total_score = 0

        for seed in afc_df.index:
            team_acronym = afc_df.loc[seed, person]
            team_data = team_situation_df[team_situation_df['Team Acronym'] == team_acronym]
            win_loss_difference = team_data['Win-Loss Difference'].values[0]
            seed_position = int(seed.split()[-1])
            score_value = scores.loc[seed_position - 1, 'Scores']
            selection_score = win_loss_difference * score_value
            total_score += selection_score

        for seed in nfc_df.index:
            team_acronym = nfc_df.loc[seed, person]
            team_data = team_situation_df[team_situation_df['Team Acronym'] == team_acronym]
            win_loss_difference = team_data['Win-Loss Difference'].values[0]
            seed_position = int(seed.split()[-1])
            score_value = scores.loc[seed_position - 1, 'Scores']
            selection_score = win_loss_difference * score_value
            total_score += selection_score

        total_scores[person] = total_score

    return total_scores

total_scores = calculate_total_scores(afc_df, nfc_df, team_situation_df, scores)

scores_df = pd.DataFrame(total_scores.items(), columns=['Person', 'Total Score'])
scores_df = scores_df.sort_values(by='Total Score', ascending=False)

scores_df.index = range(1, len(scores_df) + 1)

scores_df['Total Score'] = scores_df['Total Score'].map('{:.2f}'.format)

scores_df

,Person,Total Score
1,Caver,37.10
2,Adriana,35.90
3,Diego,35.70
4,Juan Pablo,35.20
5,Alejandro,31.50
6,Boris,29.70
7,Miguel Angel,29.30
8,Fernando,21.30
9,Patricia,16.90


In [7]:
total_scores = calculate_total_scores(afc_df, nfc_df, team_situation_df, scores)

scores_df = pd.DataFrame(total_scores.items(), columns=['Person', 'Total Score'])
scores_df = scores_df.sort_values(by='Total Score', ascending=False)
scores_df['Total Score'] = scores_df['Total Score'].map('{:.2f}'.format)

team_impact_data = []

for person in afc_df.columns:
    max_adding_team = None
    max_subtracting_team = None
    max_adding_score = -float('inf')
    max_subtracting_score = float('inf')
    
    for seed in afc_df.index:
        team_acronym = afc_df.loc[seed, person]
        team_data = team_situation_df[team_situation_df['Team Acronym'] == team_acronym]
        win_loss_difference = team_data['Win-Loss Difference'].values[0]
        seed_position = int(seed.split()[-1])
        score_value = scores.loc[seed_position - 1, 'Scores']
        selection_score = win_loss_difference * score_value
        
        if selection_score > max_adding_score:
            max_adding_team = team_acronym
            max_adding_score = selection_score
        
        if selection_score < max_subtracting_score:
            max_subtracting_team = team_acronym
            max_subtracting_score = selection_score
    
    for seed in nfc_df.index:
        team_acronym = nfc_df.loc[seed, person]
        team_data = team_situation_df[team_situation_df['Team Acronym'] == team_acronym]
        win_loss_difference = team_data['Win-Loss Difference'].values[0]
        seed_position = int(seed.split()[-1])
        score_value = scores.loc[seed_position - 1, 'Scores']
        selection_score = win_loss_difference * score_value
        
        if selection_score > max_adding_score:
            max_adding_team = team_acronym
            max_adding_score = selection_score
        
        if selection_score < max_subtracting_score:
            max_subtracting_team = team_acronym
            max_subtracting_score = selection_score
    
    team_impact_data.append({
        'Person': person,
        'Best Team': max_adding_team,
        '+': max_adding_score,
        'Worst Team': max_subtracting_team,
        '-': max_subtracting_score
    })
team_impact_df = pd.DataFrame(team_impact_data)

team_impact_df

,Person,Best Team,+,Worst Team,-
0,Adriana,PHI,8.4,ATL,-0.6
1,Alejandro,PHI,7.7,GB,-1.2
2,Boris,PHI,7.7,NYG,-3.0
3,Caver,PHI,8.4,LAC,0.0
4,Diego,PHI,8.4,LAC,0.0
5,Fernando,PHI,8.4,GB,-2.0
6,Juan Pablo,KC,6.0,NYJ,0.0
7,Miguel Angel,PHI,8.4,CAR,-4.2
8,Patricia,PHI,8.4,NE,-3.0
